In [100]:
from pyhive import hive

host_name = "localhost"
port = 10000  #default is 10000
user = "root" # user name mysql 
password = "password" # pass mysql
database="default"

conn = hive.Connection(host=host_name, port=port, username=user,password=password,
                       database=database, auth='CUSTOM')
def run_query(query):
    cur = conn.cursor()
    cur.execute(query)
    result = cur.fetchall()
    return result

In [2]:
!source ~/.bashrc

In [16]:
!~/hadoop/bin/hadoop dfs -put rows.csv /


2025-03-03 23:41:48,960 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2025-03-03 23:41:49,469 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2025-03-03 23:41:49,876 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2025-03-03 23:41:50,221 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2025-03-03 23:41:50,616 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2025-03-03 23:41:51,084 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2025-03-03 23:41:51,602 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false
2025-03-03 23:41:52,126 IN

In [18]:
!~/hadoop/bin/hadoop dfs -ls /


Found 12 items
-rw-r--r--   1 hadoop supergroup      85834 2025-01-20 15:17 /27045.txt
drwxr-xr-x   - hadoop supergroup          0 2025-02-10 14:39 /data_hive
-rw-r--r--   1 hadoop supergroup         50 2025-03-03 14:58 /demo2.txt
drwxr-xr-x   - hadoop supergroup          0 2025-02-04 20:40 /gutenberg
drwxr-xr-x   - hadoop supergroup          0 2025-02-17 13:46 /hbase_data
drwxr-xr-x   - hadoop supergroup          0 2025-03-03 14:31 /movies_greater_than_four
drwxr-xr-x   - hadoop supergroup          0 2025-03-03 14:24 /pig_data
drwx-w----   - hadoop supergroup          0 2025-03-03 15:12 /tmp
drwxr-xr-x   - hadoop supergroup          0 2025-02-10 14:54 /user1
drwxr-xr-x   - hadoop supergroup          0 2025-02-04 20:42 /yellow_trip_output
-rw-r--r--   1 hadoop supergroup 1708674492 2025-02-03 15:32 /yellow_tripdata_2016-01.csv
-rw-r--r--   1 hadoop supergroup  431701740 2025-02-03 14:12 /yellow_tripdata_2016-01.csv.gz


In [ ]:
query_drop_table = "DROP TABLE IF EXISTS yellow_taxi_2016;"
run_query(query_drop_table)

In [ ]:
query_create_table = """
CREATE TABLE IF NOT EXISTS yellow_taxi_2016 (
    VendorID INT,
    tpep_pickup_datetime STRING,
    tpep_dropoff_datetime STRING,
    passenger_count INT,
    trip_distance DOUBLE,
    pickup_longitude DOUBLE,
    pickup_latitude DOUBLE,
    RatecodeID INT,
    store_and_fwd_flag STRING,
    dropoff_longitude DOUBLE,
    dropoff_latitude DOUBLE,
    payment_type INT,
    fare_amount DOUBLE,
    extra DOUBLE,
    mta_tax DOUBLE,
    tip_amount DOUBLE,
    tolls_amount DOUBLE,
    improvement_surcharge DOUBLE,
    total_amount DOUBLE,
    PULocationID INT,
    DOLocationID INT
)
ROW FORMAT DELIMITED
FIELDS TERMINATED BY ','
TBLPROPERTIES("skip.header.line.count"="1")
"""
cur = conn.cursor()
cur.execute(query_create_table)

In [5]:
query_load_data = "load data inpath 'hdfs://localhost:9000/rows.csv' INTO TABLE yellow_taxi_2016"
cur = conn.cursor()
cur.execute(query_load_data)

In [108]:
# 1) Location with most pickups in Jan 2016
query1 = """
SELECT ROUND(pickup_latitude, 5) AS rounded_lat, 
       ROUND(pickup_longitude, 5) AS rounded_lon, 
       COUNT(*) AS pickup_count
FROM yellow_taxi_2016
WHERE FROM_UNIXTIME(UNIX_TIMESTAMP(tpep_pickup_datetime, 'MM/dd/yyyy hh:mm:ss a')) LIKE '2016-01-%'
AND pickup_latitude <> 0 AND pickup_longitude <> 0
GROUP BY ROUND(pickup_latitude, 5), ROUND(pickup_longitude, 5)
ORDER BY pickup_count DESC
LIMIT 1
"""

# 2) Day of the week with most trips in Jan 2016
query2 = """
SELECT 
    FROM_UNIXTIME(UNIX_TIMESTAMP(tpep_pickup_datetime, 'MM/dd/yyyy hh:mm:ss a'), 'EEEE') AS day_of_week,
    COUNT(*) AS trip_count
FROM yellow_taxi_2016
WHERE FROM_UNIXTIME(UNIX_TIMESTAMP(tpep_pickup_datetime, 'MM/dd/yyyy hh:mm:ss a')) LIKE '2016-01-%'
GROUP BY FROM_UNIXTIME(UNIX_TIMESTAMP(tpep_pickup_datetime, 'MM/dd/yyyy hh:mm:ss a'), 'EEEE')
ORDER BY trip_count DESC
LIMIT 1
"""

# 3) Average distance for all trips in 2016
query3 = "SELECT AVG(trip_distance) AS avg_trip_distance FROM yellow_taxi_2016"

# 4) Top 5 frequent pickup locations in 2016
query4 = """
SELECT ROUND(pickup_latitude, 5) AS rounded_lat, 
       ROUND(pickup_longitude, 5) AS rounded_lon, 
       COUNT(*) AS pickup_count
FROM yellow_taxi_2016
WHERE pickup_latitude <> 0 AND pickup_longitude <> 0
GROUP BY ROUND(pickup_latitude, 5), ROUND(pickup_longitude, 5)
ORDER BY pickup_count DESC
LIMIT 5
"""

In [102]:
result1 = run_query(query1)
print(result1)

[(40.821, -73.95467, 1023)]


In [107]:
result2 = run_query(query2)
print(result2)

[('Friday', 1972597)]


In [104]:
result3 = run_query(query3)
print(result3)

[(4.768565009352323,)]


In [109]:
result4 = run_query(query4)
print(result4)

[(40.821, -73.95467, 1524), (40.64539, -73.7767, 1308), (40.64537, -73.7767, 1254), (40.77377, -73.8709, 1208), (40.77375, -73.8709, 1067)]
